# Notebook to build code to extract words from a file 

### Since we only want words I think regex will be the best bet

In [1]:
import re
from nltk.corpus import words

In [2]:
filename = './alice30.txt'

In [3]:
#create a library of common words to detect if hyphen break accross lines needs to be merged
words_lib = set([]) 
[words_lib.add(word.lower()) for word in words.words()]
None

In [4]:
def DetectAndFixHyphenBreaks(text):
    global words_lib
    matches = re.findall(r'[a-z]+\s*\-\s*\n+\s*[a-z]+', text)
    for match in matches:
        word_1, word_2 = re.findall(r'[a-z]+', match)
        if (word_1 not in words_lib) or (word_2 not in words_lib):
            text = re.sub(match, word_1 + word_2, text)
    return text

In [5]:
def GetWordsFromFile(filename):
    #read in data
    data = open(filename, 'r')
    text = data.read().lower()
    #find words separated by a hyphen and a newline
    text = DetectAndFixHyphenBreaks(text)
    #extract words
    words = re.findall(r'[a-z]+(?:(?:\-|\')[a-z]+)?', text)
    text = None
    word_dict = {}
    for word in words:
        if word in word_dict:
            word_dict[word] = word_dict[word] + 1
        else:
            word_dict[word] = 1    
    return word_dict

In [6]:
#pass the dict of words and write the file
def WriteWordsToFile(filename, word_dict):
    data = open(filename, 'w')
    for word in word_dict:
        entry = '%s,%i\n' % (word, word_dict[word])
        data.write(entry)
    data.close()
    return None

In [7]:
word_dict = GetWordsFromFile(filename)

In [8]:
WriteWordsToFile('./words.txt', word_dict)

In [9]:
word_dict

{'secondly': 2,
 'pardon': 6,
 'saves': 1,
 'knelt': 1,
 'four': 6,
 'sleep': 6,
 'hanging': 3,
 'ringlets': 2,
 'oldest': 1,
 'hate': 2,
 'assembled': 2,
 'forget': 2,
 'whose': 2,
 'lory': 7,
 'paris': 2,
 'blacking': 1,
 'presents': 2,
 'under': 16,
 'inwards': 1,
 'sorry': 1,
 'worth': 4,
 'music': 3,
 'rise': 1,
 'every': 12,
 'seals': 1,
 'fireplace': 1,
 'school': 5,
 'prize': 1,
 'wooden': 1,
 'pinch': 2,
 'persisted': 2,
 'oblong': 1,
 'writing-desk': 1,
 'favoured': 1,
 'leaders': 1,
 'tired': 7,
 'feathers': 1,
 'elegant': 1,
 'likely': 5,
 'louder': 1,
 'machines': 1,
 'shining': 1,
 'even': 19,
 'meekly': 2,
 'hide': 1,
 'pace': 1,
 'solemn': 3,
 'thunder': 1,
 'near': 15,
 'poison': 3,
 'above': 3,
 'conduct': 1,
 'rat-hole': 1,
 'new': 5,
 'ever': 21,
 'told': 6,
 'half-past': 2,
 'never': 47,
 'wrapping': 1,
 'here': 51,
 'met': 3,
 'protection': 1,
 'hers': 4,
 'cardboard': 1,
 'shriek': 5,
 'rabbit-hole': 4,
 'daughter': 1,
 'leaves': 6,
 'changed': 8,
 'mustard-mine'

Testing the hyphen end of line checker with some junk input

In [17]:
text ='hello my dear how are you - \n\n doing. I am are fine and yourself? Sup- \n posedly we are going...'

In [18]:
DetectAndFixHyphenBreaks(text)

'hello my dear how are you - \n\n doing. I am are fine and yourself? Supposedly we are going...'

Test chunking

In [20]:
def ReadChunks(file_object, chunk_size=1024):
    #this is a nice generator to read in only a manageable sized chunk at a time
    #borrowed from http://stackoverflow.com/questions/519633/lazy-method-for-reading-big-file-in-python
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


f = open(filename)
data = []
for piece in read_in_chunks(f):
    data.append(piece)

In [21]:
data

["                ALICE'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,'\nthought Alice `without pictures or conversation?'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!

## Demonstration of 'tries' data structure usage

In [10]:
#convert the dictionary of words with counts into a tries data structure for rapid prefix searching
def ConvertWordDictToTries(word_dict):
    tries = {}
    for word in word_dict:
        count = word_dict[word]
        tries = AddWordToTries(word, tries, count)
    return tries

In [14]:
#adds a single word to the tries parse tree using a recursive scheme
def AddWordToTries(word, word_tries, count):
    letter = word[0]
    if letter in word_tries:
        sub_tries = word_tries[letter][1]
        if len(word) == 1:
            word_tries[letter][0] = count
        if len(word) > 1:
            word_tries[letter][1] = AddWordToTries(word[1:], sub_tries, count)
        return word_tries
    else:
        if len(word) == 1:
            word_tries[letter] = [count, {}]
        else:
            word_tries[letter] = [0, {}]
        sub_tries = word_tries[letter][1]
        if len(word) > 1:
            word_tries[letter][1] = AddWordToTries(word[1:], sub_tries, count)
        return word_tries

**Normal word dictionary with counts**

In [15]:
print word_dict

{'secondly': 2, 'pardon': 6, 'saves': 1, 'knelt': 1, 'four': 6, 'sleep': 6, 'hanging': 3, 'ringlets': 2, 'oldest': 1, 'hate': 2, 'assembled': 2, 'forget': 2, 'whose': 2, 'lory': 7, 'paris': 2, 'blacking': 1, 'presents': 2, 'under': 16, 'inwards': 1, 'sorry': 1, 'worth': 4, 'music': 3, 'rise': 1, 'every': 12, 'seals': 1, 'fireplace': 1, 'school': 5, 'prize': 1, 'wooden': 1, 'pinch': 2, 'persisted': 2, 'oblong': 1, 'writing-desk': 1, 'favoured': 1, 'leaders': 1, 'tired': 7, 'feathers': 1, 'elegant': 1, 'likely': 5, 'louder': 1, 'machines': 1, 'shining': 1, 'even': 19, 'meekly': 2, 'hide': 1, 'pace': 1, 'solemn': 3, 'thunder': 1, 'near': 15, 'poison': 3, 'above': 3, 'conduct': 1, 'rat-hole': 1, 'new': 5, 'ever': 21, 'told': 6, 'half-past': 2, 'never': 47, 'wrapping': 1, 'here': 51, 'met': 3, 'protection': 1, 'hers': 4, 'cardboard': 1, 'shriek': 5, 'rabbit-hole': 4, 'daughter': 1, 'leaves': 6, 'changed': 8, 'mustard-mine': 1, 'swam': 5, "i'd": 11, 'smoke': 1, "i'm": 59, 'twenty-four': 2, '

**Convert normal word dictionary with counts to a tries format**

In [16]:
#make the tries structure
tries = ConvertWordDictToTries(word_dict)

In [376]:
print tries

{'a': [631, {'c': [0, {'h': [0, {'e': [1, {}]}], 'c': [0, {'i': [0, {'d': [0, {'e': [0, {'n': [0, {'t': [2, {'a': [0, {'l': [0, {'l': [0, {'y': [1, {}]}]}]}]}]}]}]}]}], 'e': [0, {'p': [0, {'t': [0, {'a': [0, {'n': [0, {'c': [0, {'e': [1, {}]}]}]}]}]}]}], 'u': [0, {'s': [0, {'a': [0, {'t': [0, {'i': [0, {'o': [0, {'n': [1, {}]}]}]}]}], 't': [0, {'o': [0, {'m': [0, {'e': [0, {'d': [1, {}]}]}]}]}]}]}], 'o': [0, {'u': [0, {'n': [0, {'t': [1, {'i': [0, {'n': [0, {'g': [1, {}]}]}], 's': [1, {}]}]}]}]}]}], 'r': [0, {'o': [0, {'s': [0, {'s': [5, {}]}]}]}], 't': [1, {'u': [0, {'a': [0, {'l': [0, {'l': [0, {'y': [1, {}]}]}]}]}]}]}], 'b': [0, {'i': [0, {'d': [0, {'e': [1, {}]}]}], 's': [0, {'u': [0, {'r': [0, {'d': [2, {}]}]}], 'e': [0, {'n': [0, {'c': [0, {'e': [1, {}]}]}]}]}], 'l': [0, {'e': [1, {}]}], 'o': [0, {'u': [0, {'t': [94, {}]}], 'v': [0, {'e': [3, {}]}]}]}], 'd': [0, {'a': [1, {}], 'j': [0, {'o': [0, {'u': [0, {'r': [0, {'n': [1, {}]}]}]}]}], 'd': [0, {'i': [0, {'n': [0, {'g': [1, {}]

In [17]:
#I use this just to be careful and make sure no objects are changed outside of function
from copy import deepcopy

In [18]:
#recursively crawl the tries tree building back up a simple count dictionary
def ExtractRemainders(prefix, tries, word_dict, prev_letters):
    for letter in tries:
        sub_tries = deepcopy(tries[letter][1])
        if tries[letter][0] > 0:
            key = prefix + prev_letters + letter
            word_dict[key] = tries[letter][0]
        new_letters = prev_letters + letter    
        word_dict = ExtractRemainders(prefix, sub_tries, deepcopy(word_dict), new_letters)
    return word_dict

In [19]:
#extract words and counts
def MatchPrefixInTries(prefix, tries):
    #first extract the relevant subtree based on prefix
    prefix_letters = [letter for letter in prefix]
    #only use the portion of the tries tree that is needed
    sub_tries = tries
    while prefix_letters:
        letter = prefix_letters.pop(0)
        if letter in sub_tries:
            sub_tries = sub_tries[letter][1]
        else:
            return {}
    #function to parse tries tree and return valid remainders
    word_dict = {}
    word_dict = ExtractRemainders(prefix, sub_tries, {}, '')
    return word_dict

**Search tries using prefix and return a partial cound dictionary**

In [20]:
prefix_word_dict = MatchPrefixInTries('an', tries)

In [21]:
print prefix_word_dict

{'and': 866, 'ann': 4, 'angrily': 9, 'another': 22, 'anger': 2, 'any': 39, "animal's": 1, 'ancient': 1, 'angry': 5, 'answered': 4, 'animal': 1, 'answer': 9, 'and-butter': 1, 'answers': 1, 'annoy': 1, 'animals': 4, 'anything': 20, 'anxiously': 14, 'anxious': 3, 'anywhere': 1, 'antipathies': 1, 'annoyed': 1}


In [22]:
prefix_word_dict = MatchPrefixInTries('hel', tries)

In [23]:
print prefix_word_dict

{'helped': 1, 'held': 4, 'helpless': 1, 'help': 9}
